# For figure TS13

Temperature response to forcing 1750-2100

In [1]:
import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as pl
import os
from matplotlib import gridspec, rc
from matplotlib.lines import Line2D
import matplotlib.patches as mp
from netCDF4 import Dataset
import warnings

from ar6.utils.h5 import *

In [2]:
# TODO - sort out figure sizing

pl.rcParams['figure.figsize'] = (18/2.54, 11/2.54)
pl.rcParams['font.size'] = 11
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['xtick.direction'] = 'out'
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['ytick.major.left'] = True
pl.rcParams['ytick.major.size'] = 0
pl.rcParams['xtick.top'] = True
pl.rcParams['figure.dpi'] = 150

In [3]:
results = load_dict_from_hdf5('../data_output_large/twolayer_SSPs.h5')

AttributeError: 'Dataset' object has no attribute 'value'

In [4]:
results.keys()

dict_keys(['ssp119', 'ssp119_climuncert', 'ssp119_remove_co2', 'ssp119_remove_co2_climuncert', 'ssp119_remove_other_anthro', 'ssp119_remove_other_anthro_climuncert', 'ssp119_remove_other_ghg', 'ssp119_remove_other_ghg_climuncert', 'ssp126', 'ssp126_climuncert', 'ssp126_remove_co2', 'ssp126_remove_co2_climuncert', 'ssp126_remove_other_anthro', 'ssp126_remove_other_anthro_climuncert', 'ssp126_remove_other_ghg', 'ssp126_remove_other_ghg_climuncert', 'ssp245', 'ssp245_climuncert', 'ssp245_remove_co2', 'ssp245_remove_co2_climuncert', 'ssp245_remove_other_anthro', 'ssp245_remove_other_anthro_climuncert', 'ssp245_remove_other_ghg', 'ssp245_remove_other_ghg_climuncert', 'ssp370', 'ssp370_climuncert', 'ssp370_remove_co2', 'ssp370_remove_co2_climuncert', 'ssp370_remove_other_anthro', 'ssp370_remove_other_anthro_climuncert', 'ssp370_remove_other_ghg', 'ssp370_remove_other_ghg_climuncert', 'ssp585', 'ssp585_climuncert', 'ssp585_remove_co2', 'ssp585_remove_co2_climuncert', 'ssp585_remove_other_anth

In [5]:
results['ssp245']['surface_temperature'].shape

(351, 2565)

In [8]:
results['ssp245']['surface_temperature'][269].mean()

1.1905754111653173

In [12]:
forcings = ['co2', 'other_ghg', 'other_anthro']
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
for scenario in scenarios:
    for forcing in forcings:
        print(scenario, forcing, (results[scenario]['surface_temperature'][331:351] - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][331:351]).mean())

ssp119 co2 1.178977620855856
ssp119 other_ghg 0.5211981166754862
ssp119 other_anthro -0.2800433081055285
ssp126 co2 1.5030642318047547
ssp126 other_ghg 0.5371340668081277
ssp126 other_anthro -0.26266203138022476
ssp245 co2 2.193022393047796
ssp245 other_ghg 0.8465883643476835
ssp245 other_anthro -0.33760202458190014
ssp370 co2 2.9069844456625544
ssp370 other_ghg 1.354745877754714
ssp370 other_anthro -0.6881821951858071
ssp585 co2 3.545846238569255
ssp585 other_ghg 1.235982213378546
ssp585 other_anthro -0.4018650163536325


In [18]:
(results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][331:351] - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][0]).shape

(20, 2565)

In [42]:
forcings = ['co2', 'other_ghg', 'other_anthro']

AR6_forc = {}
AR6_ecsforc = {}
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp585']

base_periods = {
    '1750': 0,
    '1850-1900': slice(100,151),
    '1995-2014': slice(245,265),
    '2010-2019': slice(260,270)
}

for scenario in scenarios:
    AR6_forc[scenario] = {}
    AR6_ecsforc[scenario] = {}
    for forcing in forcings:
        AR6_forc[scenario][forcing] = {}
        AR6_ecsforc[scenario][forcing] = {}
        for base_period in base_periods:
            AR6_forc[scenario][forcing][base_period] = np.zeros(5)
            AR6_forc[scenario][forcing][base_period] = np.percentile(
                (
                    (results[scenario]['surface_temperature'][331:351].mean(axis=0) - results[scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0))-
                    (results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][331:351].mean(axis=0) - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][base_periods[base_period]].mean(axis=0))
                ), (5,16,50,84,95)
            )
            AR6_ecsforc[scenario][forcing][base_period] = np.zeros(5)
            AR6_ecsforc[scenario][forcing][base_period] = np.percentile(
                (
                    (results['%s_climuncert' % scenario]['surface_temperature'][331:351].mean(axis=0) - results['%s_climuncert' % scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0))-
                    (results['%s_remove_%s_climuncert' % (scenario, forcing)]['surface_temperature'][331:351].mean(axis=0) - results['%s_remove_%s_climuncert' % (scenario, forcing)]['surface_temperature'][base_periods[base_period]].mean(axis=0))
                ), (5,16,50,84,95)
            )
    AR6_forc[scenario]['anthro'] = {}
    AR6_ecsforc[scenario]['anthro'] = {}
    for base_period in base_periods:
        AR6_forc[scenario]['anthro'][base_period] = np.percentile(results[scenario]['surface_temperature'][331:351].mean(axis=0) - results[scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0), (5,16,50,84,95))
        AR6_ecsforc[scenario]['anthro'][base_period] = np.percentile(results['%s_climuncert' % scenario]['surface_temperature'][331:351].mean(axis=0) - results['%s_climuncert' % scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0), (5,16,50,84,95))

In [43]:
print(AR6_ecsforc['ssp245']['anthro']['1750'])
print(AR6_ecsforc['ssp245']['co2']['1750'])
print(AR6_ecsforc['ssp245']['other_ghg']['1750'])
print(AR6_ecsforc['ssp245']['other_anthro']['1750'])

[1.98667701 2.22667137 2.66397636 3.26794707 3.73775303]
[1.56737665 1.77822293 2.16257295 2.7069718  3.13771792]
[0.58913202 0.66669288 0.8282277  1.06472218 1.24725346]
[-0.69792954 -0.50580782 -0.31784233 -0.19988494 -0.14371631]


In [44]:
chapter4_tas = {}
varnames = {
    '05': 'Q05',
    'central': 'mean',
    '95': 'Q95'
}
for scenario in scenarios:
    chapter4_tas[scenario] = {}
    for pc in ['05','central','95']:
        nc = Dataset('../data_input/chapter4_assessed_ranges/assessed_%s_%s.nc' % (scenario, pc))
        chapter4_tas[scenario][pc] = nc.variables[varnames[pc]][:]
        #if pc=='95' and scen=='ssp119':
            #print(nc.variables['time'])
            #print(nc.variables['time'][:])
            #print(chapter4_tas[scen][pc])
        nc.close()
    print(scenario, chapter4_tas[scenario]['05'][-1], chapter4_tas[scenario]['central'][-1], chapter4_tas[scenario]['95'][-1])

ssp119 0.24225383730895733 0.5643103232211051 0.9575484813364052
ssp126 0.506950733053811 0.9043159359522661 1.4826491059645592
ssp245 1.243934996453968 1.814827243577203 2.585020146104215
ssp370 1.997464504676968 2.7563576355060757 3.745606165340795
ssp585 2.4421390507522727 3.5006266858546886 4.817334752006395


In [45]:
for scenario in scenarios:
    print(scenario, AR6_ecsforc[scenario]['anthro']['1995-2014'][[0,2,4]])

ssp119 [0.25073687 0.57225264 1.14241097]
ssp126 [0.52615151 0.92387796 1.62099269]
ssp245 [1.25551934 1.84064417 2.8150814 ]
ssp370 [1.99659133 2.71999932 3.80112342]
ssp585 [2.53961149 3.51098618 5.08651981]


In [54]:
output = []
for scenario in scenarios:
    for base_period in base_periods:
        for forcing in ['co2', 'other_ghg', 'other_anthro', 'anthro']:
            output.append([scenario, forcing, base_period, 
                           AR6_ecsforc[scenario][forcing][base_period][0],
                           AR6_ecsforc[scenario][forcing][base_period][1],
                           AR6_ecsforc[scenario][forcing][base_period][2],
                           AR6_ecsforc[scenario][forcing][base_period][3],
                           AR6_ecsforc[scenario][forcing][base_period][4],
                        ])

    output.append([scenario, 'chapter4', '1995-2014', chapter4_tas[scenario]['05'][-1],
                   np.nan, 
                   chapter4_tas[scenario]['central'][-1],
                   np.nan,
                   chapter4_tas[scenario]['95'][-1]])
df = pd.DataFrame(output, columns=['scenario', 'forcing', 'base_period', 'p05','p16','p50','p84','p95'])
df.to_csv('../data_output/ts_warming_ranges.csv')